In [1]:
@time begin 
    using Pkg 
    using DataFrames 
    using Pandas 
    using Distributions
    using StatsPlots 
    using PyCall

    # For module reloading
    try
        using Revise
    catch e
        @warn "Error initializing Revise" exception=(e, catch_backtrace())
    end

    # Change defaults for jupyter viz of dataframe cols/rows
    ENV["columns"] = 1000
    ENV["lines"] = 1000
end 

 25.567138 seconds (22.39 M allocations: 1.390 GiB, 2.82% gc time)


1000

In [2]:
@time begin 
    scriptdir = cd(pwd, "../..") 
    PyCall.pushfirst!(PyVector(pyimport("sys")["path"]), scriptdir)
    energy_survey = pyimport("astrobot.energy_survey")
    es = energy_survey.EnergySurvey()
    println(scriptdir, "\n")
end 

C:\Users\saera\master\astrobot

  0.060001 seconds (22.68 k allocations: 1.296 MiB, 56.87% compilation time)


In [3]:
begin
    dd = es.com_survey_df()
    kk = es.com_key_df()

    # TODO: initial data cleaning
    # dd = dd.query("vacant != 1 and monuse > 10")
    # dd = dd.query("yrcon > 2000") 
    dd = es.loads(dd)

    # Convert to julia df
    dd = DataFrames.DataFrame(Pandas.DataFrame(dd));
    kk = DataFrames.DataFrame(Pandas.DataFrame(kk));
    println("")
end 

In [4]:
begin 
    regions = kk[3, 7]
    bldtypes = kk[4, 7] 

    println(regions)
    println("---\n")
    println(bldtypes)
    println("---\n")
end

'1' = 'New England'
'2' = 'Middle Atlantic'
'3' = 'East North Central'
'4' = 'West North Central'
'5' = 'South Atlantic'
'6' = 'East South Central'
'7' = 'West South Central'
'8' = 'Mountain'
'9' = 'Pacific'
---

'01' = 'Vacant'
'02' = 'Office'
'04' = 'Laboratory'
'05' = 'Nonrefrigerated warehouse'
'06' = 'Food sales'
'07' = 'Public order and safety'
'08' = 'Outpatient health care'
'11' = 'Refrigerated warehouse'
'12' = 'Religious worship'
'13' = 'Public assembly'
'14' = 'Education'
'15' = 'Food service'
'16' = 'Inpatient health care'
'17' = 'Nursing'
'18' = 'Lodging'
'23' = 'Strip shopping mall'
'24' = 'Enclosed mall'
'25' = 'Retail other than mall'
'26' = 'Service'
'91' = 'Other'
---



In [5]:
# :pba = 2 office
# :region = 1 New England
kk[1:5,2]

5-element Vector{String}:
 "PUBID"
 "REGION"
 "CENDIV"
 "PBA"
 "FREESTN"

In [113]:
begin 
    # q_arr: all t, h, wts
    _data = sort!(dd[:, [:heat, :pba, :region, :finalwt]])
    N_finalwt = sum(_data[:, :finalwt])
    N = size(_data)[1]
    @show _data[1:5, :]
    println("# of bldings = ", N_finalwt);
    println("# of idx = ", N);
    #q_arr[1:5, :]
end 

_data[1:5, :] = 5×4 DataFrame
 Row │ heat     pba    region  finalwt
     │ Float64  Int64  Int64   Float64
─────┼──────────────────────────────────
   1 │     0.0      1       1   146.021
   2 │     0.0      1       1   327.511
   3 │     0.0      1       1   575.39
   4 │     0.0      1       1  1145.08
   5 │     0.0      1       1  1145.08
# of bldings = 5.557138453172112e6
# of idx = 6720


In [114]:
# function mquery(dt::Any; query::String="", inmod::Union{Nothing, Module}=Nothing)
#     if inmod isa Module 
#         method_arr = methodswith(typeof(dt), inmod, supertypes=true)
#     else
#         method_arr = methodswith(typeof(dt), supertypes=true)
#     end
#     return [m for m in method_arr if contains(String(m.name), query)]
#     end 

function mquery(dt::Any, query::String="")
    method_arr = methodswith(typeof(dt), supertypes=true)
    return [m for m in method_arr if contains(String(m.name), query)]
    end 
    
function mquery(dt::Any, inmod::Module, query::String="")
    method_arr = methodswith(typeof(dt), inmod, supertypes=true)
    return [m for m in method_arr if contains(String(m.name), query)]
    end 


struct X
    x::Int32
end 
y(X) = 3 * X.x
x = X(2)
y(x)
@show typeof(X);
@show fieldnames(X);  # properties defined in constructor
@show propertynames(X);  # datatype properties
println(X.mutable)
methodswith(typeof(X));  # functions you can use w/ type
meth = mquery(_data, "group")

typeof(X) = DataType
fieldnames(X) = (:x,)
propertynames(X) = (:name, :super, :parameters, :types, :names, :instance, :layout, :size, :ninitialized, :hash, :abstract, :mutable, :hasfreetypevars, :isconcretetype, :isdispatchtuple, :isbitstype, :zeroinit, :isinlinealloc, :has_concrete_subtype, :cached_by_hash)
false


[1] groupby(df::AbstractDataFrame, cols; sort, skipmissing) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\groupeddataframe.jl:207

In [122]:
eui_sort = reverse(sort(_data[:, :heat]))
println("eui range: ", eui_sort[1], " - ", eui_sort[N])
#eui_bins = [0:5:1500;] # histogram bins from 0 - 1500 in inc=5
# Not very efficient, but good enough...
eui_bins = zeros(N)
for i in range(1, length=N)
    _eui = _data[i, :heat]
    eui_bins[i] = _eui - (_eui%5)
end
_data[!, "bin_heat"] = eui_bins;
last(_data[:, [:bin_heat, :heat, :pba, :region, :finalwt]], 5)

eui range: 1023.0853846153847 - 0.0


,bin_heat,heat,pba,region,finalwt
,Float64,Float64,Int64,Int64,Float64
1,455.0,455.43,16,1,2.65274
2,460.0,461.653,26,3,1162.26
3,470.0,474.796,16,1,2.65274
4,845.0,849.353,13,3,11.705
5,1020.0,1023.09,91,1,1868.4


In [124]:
# :pba = 4 office
# :region = 1 New England
# Check if P(EUI | Office, New England) == P(EUI | Office | New England)
# P(E | T) = P(E) P(T | E) / P(T)

"""
    create_pmf(data, col)

Creates pmf as N x 2 matrix with quantities and associated probability.
"""
function create_pmf(data_df::DataFrames.DataFrame, col::String)    
    groupdf = DataFrames.groupby(data_df, [col]);
    N_groups = length(groupdf);
    println(summary(groupdf));
    p_pba = [sum(_df[:, :finalwt]) for _df in groupdf]
    k_pba = [_df[1, col] for _df in groupdf]  # get key for each group
    pmf_pba = hcat(k_pba, p_pba / sum(p_pba))  # normalize
    @assert abs(sum(pmf_pba[:, 2]) - 1.0) < 1e-10 "pmf does not sum to 1"
    return pmf_pba
end 

pmf_pba = create_pmf(_data, "pba");
pmf_reg = create_pmf(_data, "region");
pmf_heat = create_pmf(_data, "bin_heat");

GroupedDataFrame with 20 groups based on key: pba
GroupedDataFrame with 4 groups based on key: region
GroupedDataFrame with 67 groups based on key: bin_heat


In [104]:
# TODO: create pmf multiplication: pmfA * pmfB ????

In [24]:
mquery(groupdf)

[1] getindex(gd::GroupedDataFrame, idxs::Colon) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\groupeddataframe.jl:460
[2] getindex(gd::GroupedDataFrame, idxs::AbstractVector{var"#s285"} where var"#s285"<:Integer) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\groupeddataframe.jl:441
[3] getindex(gd::GroupedDataFrame, idx::Integer) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\groupeddataframe.jl:437
[4] getindex(gd::GroupedDataFrame, idx...) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\groupeddataframe.jl:651
[5] get(gd::GroupedDataFrame, key::Union{Tuple, AbstractDict{Symbol, V} where V, AbstractDict{var"#s285", V} where {var"#s285"<:AbstractString, V}, NamedTuple, DataFrames.GroupKey}, default) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\groupeddataframe.jl:943
[6] combine(f::Union{Function, Type}, gd::GroupedDataFrame; keepkeys, ungroup, renamecols) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\splitapplycombine.jl:690
[7] combine(f::Pair, gd::GroupedDataFrame; keepkeys, ungroup, renamecols) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\splitapplycombine.jl:699
[8] combine(gd::GroupedDataFrame, cs::Union{Regex, AbstractString, Function, Signed, Symbol, Unsigned, Pair, AbstractVector{T} where T, Type, All, Between, Cols, InvertedIndex, AbstractVecOrMat{var"#s428"} where var"#s428"<:Pair}...; keepkeys, ungroup, renamecols) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\splitapplycombine.jl:707
[9] groupcols(gd::GroupedDataFrame) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\groupeddataframe.jl:362
[10] groupindices(gd::GroupedDataFrame) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\groupeddataframe.jl:355
[11] select(f::Union{Function, Type}, gd::GroupedDataFrame; copycols, keepkeys, ungroup, renamecols) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\splitapplycombine.jl:711
[12] select(gd::GroupedDataFrame, args::Union{Regex, AbstractString, Function, Signed, Symbol, Unsigned, Pair, AbstractVector{T} where T, Type, All, Between, Cols, InvertedIndex, AbstractVecOrMat{var"#s428"} where var"#s428"<:Pair}...; copycols, keepkeys, ungroup, renamecols) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\splitapplycombine.jl:722
[13] select!(f::Union{Function, Type}, gd::GroupedDataFrame; ungroup, renamecols) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\splitapplycombine.jl:745
[14] select!(gd::GroupedDataFrame{DataFrames.DataFrame}, args::Union{Regex, AbstractString, Function, Signed, Symbol, Unsigned, Pair, AbstractVector{T} where T, Type, All, Between, Cols, InvertedIndex, AbstractVecOrMat{var"#s428"} where var"#s428"<:Pair}...; ungroup, renamecols) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\splitapplycombine.jl:752
[15] subset(gdf::GroupedDataFrame, args...; skipmissing, view, ungroup) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\abstractdataframe\subset.jl:177
[16] subset!(gdf::GroupedDataFrame, args...; skipmissing, ungroup) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\abstractdataframe\subset.jl:295
[17] transform(f::Union{Function, Type}, gd::GroupedDataFrame; copycols, keepkeys, ungroup, renamecols) in DataFrames at C:\Users\saera\.julia\packages\DataFrames\vuMM8\src\groupeddataframe\splitapplycombine.jl:726
[18] transform(gd::GroupedDataFrame, args::Union{Regex, AbstractString, Function, Signed, Symbol, Unsigned, Pair, AbstractVector{T} where T, Type, All, Between, Cols, InvertedIndex, AbstractVecOrMat{var"#s428"} where var"#s42

In [17]:
?zip


search: zip



```
zip(iters...)
```

Run multiple iterators at the same time, until any of them is exhausted. The value type of the `zip` iterator is a tuple of values of its subiterators.

!!! note
    `zip` orders the calls to its subiterators in such a way that stateful iterators will not advance when another iterator finishes in the current iteration.


# Examples

```jldoctest
julia> a = 1:5
1:5

julia> b = ["e","d","b","c","a"]
5-element Vector{String}:
 "e"
 "d"
 "b"
 "c"
 "a"

julia> c = zip(a,b)
zip(1:5, ["e", "d", "b", "c", "a"])

julia> length(c)
5

julia> first(c)
(1, "e")
```


In [9]:
# """
# - P(100, Q_arr) = len(where(Q_arr == 100) / len(Q_arr)
# - get p(Q=100); p(z=5B); p(z=5B | Q=100)
# - p(100, Q_arr) * likeli / p(5B, z_arr)
#     - p(z_arr if Q=10 and z=5B | Q_arr if 100)
# - test 1A - 6B. Should see p get smaller.
# - cond_pdist: p(q | z & t) for all q set Q_arr
# - E(pdist) = \sum{p(q) q}
# - prob_model(z, t) = E[p(q | z & t) for all q]
# - p(q); smarter = p(q) p(z|q) / p(z)
# - p(xx) = p(q|z); smarter = p(x) p(v|x) / p(v)
#     - require p(v|x): cache x-metadata or compute raw.
#     - metadata = prob model struct.
# """